In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lfilter
from stockwell import st

In [ ]:
# SDOF Transient Response (ISO 18431-4)
# th_in : acceleration time history at base
# th_out : acceleration time history at mass
def SDOF_ISO(th_in,dt,fn,zeta):

    wn = 2*np.pi*fn;
    wd = wn*np.sqrt(1-zeta**2);
    A=wn*dt*zeta;
    B=wd*dt;
    
    a, b = np.zeros(3), np.zeros(3);

    b[0] = 1 - np.exp(-A)*np.sin(B)/B;
    b[1] = 2*np.exp(-A)*(np.sin(B)/B - np.cos(B));
    b[2] = np.exp(-2*A) - np.exp(-A) * np.sin(B)/B;

    a[0] = 1;
    a[1] = -2*np.exp(-A)*np.cos(B);
    a[2] = np.exp(-2*A);

    th_out=lfilter(b, a, th_in, axis=-1, zi=None);
    
    return th_out;

In [ ]:
## SRS (ISO 18431-4)
def srs_iso(ft,th,dr,dt):
    nf = len(ft);
    rs = np.zeros(nf);
    for i in range(nf):
        rs[i]= max(abs(SDOF_ISO(th,dt,ft[i],dr)));
    return ft,rs;

In [ ]:
def read_th(data,dt):
    th = np.zeros(5*len(data));
    tmax = dt*len(th);
    t = np.arange(dt,tmax+dt,dt);
    idx=0;
    for i in range(len(data)):
        for j in range(5):
            th[idx] = data[i,j]; idx += 1;
    return t,th

In [ ]:
def gen_ft(t):
    dt = t[1] - t[0];
    fmax = 1/2/dt;
    tmax = max(t); Nt = len(t); df = fmax/(Nt//2); fmin = df;
    freq = np.arange(df, fmax+df, df);
    return freq,fmin,fmax,df

In [ ]:
def fileI(filename):
    tmp = []
    f = open(filename,'r');
    lines = f.readlines()
    for line in lines:
        tmp.append(line);
    out=np.zeros([len(tmp),5]);
    for i in range(len(out)):
        out[i,:] = np.array(tmp[i].split()).astype(np.float64)
    return out;